In [15]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
import scipy.stats as stats
import pandas as pd
import torch as tn

Directory where the simulation output files are stored.

In [6]:
path = 'Std model simulation directory'

The function *read isochrone data* reads the data relative to MS, RGB and remnant stars contained in the isochrone for the age of 13 Gyr, and returns those of MS and RGB stars (*MS data* and *RGB data*, respectively).

In [7]:
def read_isochrone_data():

    iso_data = []

    with open('Isochrone','r') as file:
        for line in file:
            iso_data.append([float(x) for x in line.split()])
    iso_data = np.asarray(iso_data)

    label = iso_data[:,9]
    
    MS_data = iso_data[label < 3 ]
    RGB_data = iso_data[label == 3]
    remn_data = iso_data[label == 9]

    return MS_data, RGB_data

The function *luminosities* extracts both the bolometric luminosity, and the luminosities in the V and B bands for stars in different evolutionary stages (i.e., MS, RGB and WD cooling sequence) from isochrone data, in order to determine the total luminosity of the system. Hence the function returns:

1) *L$_{bol}$* : total bolometric luminosity in units of L$_\odot$ (float type);

2) *L$_V$* : total luminosity in the V band in units of L$_\odot$ (float type);

3) *L$_B$* : total luminosity in the B band in units of L$_\odot$ (float type).

In [8]:
def luminosities(N_MS, N_RGB, N_WD, MS_data, RGB_data):
    
    bol_mag_sun = 4.75
    V_mag_sun = 4.83
    B_mag_sun = 5.48
    
    # MS luminosities

    bol_mag_MS = MS_data[:,26]
    B_mag_MS = MS_data[:,28]
    V_mag_MS = MS_data[:,29]

    L_bol_MS = np.exp(-(bol_mag_MS-bol_mag_sun)/2.5)
    L_bol_tot_MS = np.sum(L_bol_MS)
    L_bol_mean_MS = L_bol_tot_MS/len(L_bol_MS)

    L_V_MS = np.exp(-(V_mag_MS-V_mag_sun)/2.5)
    L_V_tot_MS = np.sum(L_V_MS)
    L_V_mean_MS = L_V_tot_MS/len(L_V_MS)

    L_B_MS = np.exp(-(B_mag_MS-B_mag_sun)/2.5)
    L_B_tot_MS = np.sum(L_B_MS)
    L_B_mean_MS = L_B_tot_MS/len(L_B_MS)
    
    L_bol_all_MS = L_bol_mean_MS*N_MS
    L_V_all_MS = L_V_mean_MS*N_MS
    L_B_all_MS = L_B_mean_MS*N_MS    
    
    # RGB luminosities
    
    bol_mag_RGB = RGB_data[:,26]
    B_mag_RGB = RGB_data[:,28]
    V_mag_RGB = RGB_data[:,29]

    L_bol_RGB = np.exp(-(bol_mag_RGB-bol_mag_sun))/2.5
    L_bol_tot_RGB = np.sum(L_bol_RGB)
    L_bol_mean_RGB = L_bol_tot_RGB/len(L_bol_RGB)

    L_V_RGB = np.exp(-(V_mag_RGB-V_mag_sun)/2.5)
    L_V_tot_RGB = np.sum(L_V_RGB)
    L_V_mean_RGB = L_V_tot_RGB/len(L_V_RGB)

    L_B_RGB = np.exp(-(B_mag_RGB-B_mag_sun)/2.5)
    L_B_tot_RGB = np.sum(L_B_RGB)
    L_B_mean_RGB = L_B_tot_RGB/len(L_B_RGB)
    
    L_bol_all_RGB = L_bol_mean_RGB*N_RGB
    L_V_all_RGB = L_V_mean_RGB*N_RGB
    L_B_all_RGB = L_B_mean_RGB*N_RGB
    
    # WD luminosities
    
    L_WD = 10**(-4)*N_WD
    
    # Total luminosities
    
    L_bol = np.sum(L_bol_all_MS + L_bol_all_RGB + L_WD) # bolometric
    
    L_V = np.sum(L_V_all_MS + L_V_all_RGB + L_WD) # V photometric band
    
    L_B = np.sum(L_B_all_MS + L_B_all_RGB + L_WD) # B photometric band

    return L_bol, L_V, L_B    

The function *M/L ratio* computes the mass-to-light ratio by considering the dynamical mass and the total luminosity of the system, and returns:

1) *M/L$_{bol}$ ratio* : mass-to-light ratio for the total bolometric luminosity in M$_\odot$/L$_{bol,\odot}$ (float type);

2) *M/L$_V$ ratio* : mass-to-light ratio for the total V-band luminosity in M$_\odot$/L$_{V,\odot}$ (float type);

3) *M/L$_B$ ratio* : mass-to-light ratio for the total B-band luminosity in M$_\odot$/L$_{B,\odot}$ (float type).

In [9]:
def ML_ratio(M, L_bol, L_V, L_B):
    
    ML_bol_ratio = M/L_bol
    
    ML_V_ratio = M/L_V
    
    ML_B_ratio = M/L_B
    
    return ML_bol_ratio, ML_V_ratio, ML_B_ratio

The function *obtain velocity dispersion* reads the output files named *sigma$_{100f_b}$* in the simulation directory and stores their data into a 8$\times$9 matrix. Hence it returns, at varying binary fraction:

1) *$\sigma$* : observed velocity dispersion in km/s (1-D array);

2) *$\sigma_0$* : observed velocity dispersion in km/s (1-D array).

In particular, here *$\sigma$*=*$\sigma_{tot}$* and *$\sigma_0$*=*$\sigma_{sb}$*.

In [10]:
def obtain_velocity_dispersion(path):
    
    f_b = np.array([0,5,10,15,20,25,30,35,40])
    
    sigma_matrix = np.zeros(shape=(8,9))
    i = 0
    for number in f_b:
        j = 0
        with open('path/sigma_%i'%int(number), 'r') as file:
            for line in file:
                sigma_matrix[j,i]=[float(x) for x in line.split()][0]
                j +=1
        i += 1
    
    sigma = sigma_matrix[0,:]
    sigma_0 = sigma_matrix[1,:]
    
    return sigma, sigma_0

The function *obtain star numbers* reads the output files named *stars$_{100f_b}$* in the simulation directory and stores their data into a 3$\times$9 matrix. Hence it returns, at varying binary fraction:

1) *N$_{MS}$* : number of MS stars (1-D array);

2) *N$_{RGB}$* : number of RGB stars (1-D array);

3) *N$_{WD}$* : number of WDs (1-D array).

In [12]:
def obtain_star_numbers(path):
    
    f_b = np.array([0,5,10,15,20,25,30,35,40])
    
    stars_matrix = np.zeros(shape=(3,9))
    i = 0
    for number in f_b:
        j = 0
        with open('path/stars_%i'%int(number), 'r') as file:
            for line in file:
                stars_matrix[j,i]=[float(x) for x in line.split()][0]
                j +=1
        i += 1
    
    N_MS = stars_matrix[0,:]
    N_RGB = stars_matrix[1,:]
    N_WD = stars_matrix[2,:]
    
    return N_MS, N_RGB, N_WD

The function *dynamical mass* derives the dynamical mass *M$_{dyn}$* (1-D array) of the system in M$_\odot$ from the observed and the intrinsic velocity dispersion (*$\sigma$* and *$\sigma_0$*, respectively) at varying binary fraction.

In [13]:
def dynamical_mass(R, M_tot, sigma, sigma_0):
    
    normalized_delta_M = (sigma**2-sigma_0**2)/(sigma_0**2)
    
    M_dyn = (M_tot*normalized_delta_M) + M_tot
    
    return M_dyn

Functions initialization, where *R*=50 pc and *M$_{tot}$*=5$\times$10$^4$ M$_\odot$ for the simulated UFD, whereas *R*=3$\times$10$^3$ pc and *M$_{tot}$*=10$^7$ M$_\odot$ for the simulated dSph.

In [16]:
sigma, sigma_0 = obtain_velocity_dispersion(path)

N_MS, N_RGB, N_WD = obtain_star_numbers(path)

M_dyn = dynamical_mass(R, M_tot, sigma, sigma_0)

FileNotFoundError: [Errno 2] No such file or directory: 'path/sigma_0'

Values of the mass-to-light ratio for *f$_b$*=0.

In [5]:
L_bol_0, L_V_0, L_B_0 = luminosities(N_MS[0], N_RGB[0], N_WD[0], MS_data, RGB_data)

ML_bol_ratio_0, ML_V_ratio_0, ML_B_ratio_0 = ML_ratio(M_dyn[0], L_bol_0, L_V_0, L_B_0)

print('M/L$_{bol}$ for *f$_b$=0 :', ML_bol_ratio_0)
print('M/L$_V$ for *f$_b$=0 :', ML_V_ratio_0)
print('M/L$_B$ for *f$_b$=0 :', ML_B_ratio_0)

NameError: name 'luminosities' is not defined

Values of the mass-to-light ratio for *f$_b$*=0.05.

In [12]:
L_bol_5, L_V_5, L_B_5 = luminosities(N_MS[1], N_RGB[1], N_WD[1], MS_data, RGB_data)

ML_bol_ratio_5, ML_V_ratio_5, ML_B_ratio_5 = ML_ratio(M_dyn[1], L_bol_5, L_V_5, L_B_5)

print('M/L$_{bol}$ for *f$_b$=0.05 :', ML_bol_ratio_5)
print('M/L$_V$ for *f$_b$=0.05 :', ML_V_ratio_5)
print('M/L$_B$ for *f$_b$=0.05 :', ML_B_ratio_5)

Values of M/L for f_b = 0.05 (UFD): 1.946946477045229 19.042529698742246 15.647229775498234
Values of M/L for f_b = 0.05 (dSph): 0.07432608270763114 0.7269047851689707 0.5974171520369083


Values of the mass-to-light ratio for *f$_b$*=0.15.

In [13]:
L_bol_15, L_V_15, L_B_15 = luminosities(N_MS[3], N_RGB[3], N_WD[3], MS_data, RGB_data)

ML_bol_ratio_15, ML_V_ratio_15, ML_B_ratio_15 = ML_ratio(M_dyn[3], L_bol_15, L_V_15, L_B_15)

print('M/L$_{bol}$ for *f$_b$=0.15 :', ML_bol_ratio_15)
print('M/L$_V$ for *f$_b$=0.15 :', ML_V_ratio_15)
print('M/L$_B$ for *f$_b$=0.15 :', ML_B_ratio_15)

Values of M/L for f_b = 0.15 (UFD): 5.640376359145963 55.162004581688436 45.33683508216606
Values of M/L for f_b = 0.15 (dSph): 0.5797129804534176 5.669162348882274 4.6601123397374336


Values of the mass-to-light ratio for *f$_b$*=0.30.

In [14]:
L_bol_30, L_V_30, L_B_30 = luminosities(N_MS[6], N_RGB[6], N_WD[6], MS_data, RGB_data)

ML_bol_ratio_30, ML_V_ratio_30, ML_B_ratio_30 = ML_ratio(M_dyn[6], L_bol_30, L_V_30, L_B_30)

print('M/L$_{bol}$ for *f$_b$=0.30 :', ML_bol_ratio_30)
print('M/L$_V$ for *f$_b$=0.30 :', ML_V_ratio_30)
print('M/L$_B$ for *f$_b$=0.30 :', ML_B_ratio_30)

Values of M/L for f_b = 0.3 (UFD): 11.261677264720776 110.12866323325507 90.53149369848155
Values of M/L for f_b = 0.3 (dSph): 1.2925518579881279 12.64087034464634 10.389523928154837


Values of the mass-to-light ratio for *f$_b$*=0.40.

In [15]:
L_bol_40, L_V_40, L_B_40 = luminosities(N_MS[8], N_RGB[8], N_WD[8], MS_data, RGB_data)

ML_bol_ratio_40, ML_V_ratio_40, ML_B_ratio_40 = ML_ratio(M_dyn[8], L_bol_40, L_V_40, L_B_40)

print('M/L$_{bol}$ for *f$_b$=0.40 :', ML_bol_ratio_40)
print('M/L$_V$ for *f$_b$=0.40 :', ML_V_ratio_40)
print('M/L$_B$ for *f$_b$=0.40 :', ML_B_ratio_40)

Values of M/L for f_b = 0.4 (UFD): 14.918795450155246 145.88255527982778 119.9425008604154
Values of M/L for f_b = 0.4 (dSph): 1.7011511752830892 16.63747147462019 13.673094008484497


The function *plot M/L$_{bol}$ ratio* plots the mass-to-bolometric-light ratio as a function of the binary fraction *f$_b$*.

In [1]:
def plot_ML_bol_ratio(ML_bol_ratio_0, ML_bol_ratio_5, ML_bol_ratio_15, ML_bol_ratio_30, ML_bol_ratio_40, _plot=True):

    def precision(y, pos):
        return '%2.2f' % (y)

    formatter = FuncFormatter(precision)

    ML_bol_ratio = np.zeros(5)

    ML_bol_ratio[0] = ML_bol_ratio_0
    ML_bol_ratio[1] = ML_bol_ratio_5
    ML_bol_ratio[2] = ML_bol_ratio_15
    ML_bol_ratio[3] = ML_bol_ratio_30
    ML_bol_ratio[4] = ML_bol_ratio_40

    if _plot:
        
        f_b = np.array([int(0),0.05,0.15,0.3,0.4])
        fig, ay = plt.subplots(figsize=(6,6), dpi=120)
        X = f_b
        Y = ML_bol_ratio
        plt.plot(X, Y, marker='o', markersize=7, mfc='white', mec='black', color='black', linewidth=2, alpha=1)
        plt.xlabel('$f_b$', fontsize=25)
        plt.ylabel('$M/L_{bol}$ [M$_\odot$/L$_{bol,\odot}$]', fontsize=25)
        plt.xticks(ticks=X, labels=['0','0.05','0.15','0.3','0.4'], fontsize=18)
        ay.yaxis.set_major_formatter(formatter)
        plt.yticks(ticks=Y, fontsize=18)
        plt.show()
    
    return 0

The function *plot M/L$_V$ ratio* plots the mass-to-light ratio in the V band as a function of the binary fraction *f$_b$*.

In [2]:
def plot_ML_V_ratio(ML_V_ratio_0, ML_V_ratio_5, ML_V_ratio_15, ML_V_ratio_30, ML_V_ratio_40, _plot=True):

    def precision(y, pos):
        return '%2.2f' % (y)

    formatter = FuncFormatter(precision)

    ML_V_ratio = np.zeros(5)

    ML_V_ratio[0] = ML_V_ratio_0
    ML_V_ratio[1] = ML_V_ratio_5
    ML_V_ratio[2] = ML_V_ratio_15
    ML_V_ratio[3] = ML_V_ratio_30
    ML_V_ratio[4] = ML_V_ratio_40
    
    if _plot:

        f_b = np.array([int(0),0.05,0.15,0.3,0.4])
        fig, ay = plt.subplots(figsize=(6,6), dpi=120)
        X = f_b
        Y = ML_V_ratio
        plt.plot(X, Y, marker='o', markersize=7, mfc='white', mec='black', color='black', linewidth=2, alpha=1)
        plt.xlabel('$f_b$', fontsize=25)
        plt.ylabel('$M/L_V$ [M$_\odot$/L$_{V,\odot}$]', fontsize=25)
        plt.xticks(ticks=X, labels=['0','0.05','0.15','0.3','0.4'], fontsize=18)
        ay.yaxis.set_major_formatter(formatter)
        plt.yticks(ticks=Y, fontsize=18)
        plt.show()
    
    return 0

The function *plot M/L$_B$ ratio* plots the mass-to-light ratio in the B band as a function of the binary fraction *f$_b$*.

In [3]:
def plot_ML_B_ratio(ML_B_ratio_0, ML_B_ratio_5, ML_B_ratio_15, ML_B_ratio_30, ML_B_ratio_40, _plot=True):

    def precision(y, pos):
        return '%2.2f' % (y)

    formatter = FuncFormatter(precision)

    ML_V_ratio = np.zeros(5)

    ML_B_ratio[0] = ML_B_ratio_0
    ML_B_ratio[1] = ML_B_ratio_5
    ML_B_ratio[2] = ML_B_ratio_15
    ML_B_ratio[3] = ML_B_ratio_30
    ML_B_ratio[4] = ML_B_ratio_40
    
    if _plot:

        f_b = np.array([int(0),0.05,0.15,0.3,0.4])
        fig, ay = plt.subplots(figsize=(6,6), dpi=120)
        X = f_b
        Y = ML_B_ratio
        plt.plot(X, Y, marker='o', markersize=7, mfc='white', mec='black', color='black', linewidth=2, alpha=1)
        plt.xlabel('$f_b$', fontsize=25)
        plt.ylabel('$M/L_B$ [M$_\odot$/L$_{B,\odot}$]', fontsize=25)
        plt.xticks(ticks=X, labels=['0','0.05','0.15','0.3','0.4'], fontsize=18)
        ay.yaxis.set_major_formatter(formatter)
        plt.yticks(ticks=Y, fontsize=18)
        plt.show()
    
    return 0

Plot functions initialization. If plots are not required, then *_plot=False*.

In [4]:
plot_ML_bol_ratio(ML_bol_ratio_0, ML_bol_ratio_5, ML_bol_ratio_15, ML_bol_ratio_30, ML_bol_ratio_40, _plot=True)

plot_ML_V_ratio(ML_V_ratio_0, ML_V_ratio_5, ML_V_ratio_15, ML_V_ratio_30, ML_V_ratio_40, _plot=True)

plot_ML_B_ratio(ML_B_ratio_0, ML_B_ratio_5, ML_B_ratio_15, ML_B_ratio_30, ML_B_ratio_40, _plot=True)

NameError: name 'ML_bol_ratio_0' is not defined